In [1]:
%pwd

'/home/milad/projects/medical-nlp-pipeline/research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/home/milad/projects/medical-nlp-pipeline'

In [ ]:
import opendatasets as od

# Assign the Kaggle data set URL into variable
dataset = 'https://www.kaggle.com/datasets/prasad22/ca-independent-medical-review'
# Using opendatasets let's download the data sets
od.download(dataset)

In [4]:
from dataclasses import dataclass
from pathlib import Path

# specifies the type of value related to the key in yaml file
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dr: Path
    source_URL: str

In [5]:
from medical_nlp.constants import *
from medical_nlp.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, config_file_path= CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL
        )
        
        return data_ingestion_config